# Practical Statistics for Data Science

### Estructura de datos no rectangular

Estos son datos como:
- Series temporales los cuales registran mediciones sucesivas de la misma variable. Es el mmaterial ideal para pronosticos estadisticos y el componente principal del Internet de las Cosas.
- Datos espaciales utilizados en mapas y análisis de ubicación y son mas complejas y variadas que las estructuras de datos rectangulares. El foco de los datos es un objeto y sus coordenadas espaciales.
- Los grafos son estructuras de datos que se utilizan para representar relaciones fisicas, sociales y abstractas. Un ejemplo son las conexiones entre personas en la Red de Facebook.Utilizados en la optimizacion de la red  y en sistemas de recomendacion.

La estructura de datos basica in Ciencia de datos es la matriz rectangular en la cual las filas son registros y las columnas son las variables.

### Estimacion de Localizacion